In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mpl
from matplotlib import pyplot as plt

from scipy.stats import zscore

from collections import defaultdict, Counter

import scipy.spatial as sp
import scipy.cluster.hierarchy as hc

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Summary

This notebook describes how to roughly reproduce the analysis leading up to Figure 3.

In [11]:
hs_muts = pd.read_hdf("../data/hs_muts.h5",key="hs_muts")
damaging_muts = pd.read_hdf("../data/damaging_muts.h5",key="damaging_muts")

chromatin_profiling = pd.read_hdf("../data/CCLE_GlobalChromatinProfiling_20181130.hdf",key="chromatin_profiling")
mutation_calls = pd.read_hdf("../data/depmap_19Q1_mutation_calls.h5",key="mutation_calls")

In [22]:
chromatin_profiling = chromatin_profiling.dropna(axis=1,thresh=875)

In [57]:
muts_align, chromatin_align = hs_muts.align(chromatin_profiling,axis=0,join="inner")

In [79]:
crebbp = mutation_calls[mutation_calls["Hugo_Symbol"]=="CREBBP"]
crebbp[crebbp["Variant_annotation"]=="damaging"]

Hugo_Symbol  Entrez_Gene_Id  NCBI_Build Chromosome  Start_position  \
44657        CREBBP            1387          37         16         3777734   
46376        CREBBP            1387          37         16         3830732   
55016        CREBBP            1387          37         16         3786144   
61889        CREBBP            1387          37         16         3790512   
73171        CREBBP            1387          37         16         3781773   
...             ...             ...         ...        ...             ...   
1177403      CREBBP            1387          37         16         3790488   
1180341      CREBBP            1387          37         16         3779548   
1192508      CREBBP            1387          37         16         3807363   
1192509      CREBBP            1387          37         16         3817721   
1194274      CREBBP            1387          37         16         3808955   

         End_position Strand Variant_Classification Variant_Type  \
44657         3777734      +        Frame_Shift_Del          DEL   
46376         3830732      +            Splice_Site          SNP   
55016         3786144      +      Nonsense_Mutation          SNP   
61889         3790512      +      Nonsense_Mutation          SNP   
73171         3781784      +            Splice_Site          DEL   
...               ...    ...                    ...          ...   
1177403       3790488      +      Nonsense_Mutation          SNP   
1180341       3779548      +      Nonsense_Mutation          SNP   
1192508       3807364      +        Frame_Shift_Ins          INS   
1192509       3817721      +            Splice_Site          DEL   
1194274       3808955      +      Nonsense_Mutation          SNP   

        Reference_Allele  ... VA_WES_AC CGA_WES_AC SangerWES_AC  \
44657                  A  ...       NaN        NaN        42:11   
46376                  C  ...       NaN        NaN          NaN   
55016                  C  ...       NaN        NaN        59:33   
61889                  G  ...       NaN        NaN        11:30   
73171       CTACCTCCTTGT  ...       NaN        NaN         14:0   
...                  ...  ...       ...        ...          ...   
1177403                G  ...       NaN        NaN        27:35   
1180341                G  ...       NaN        NaN        65:89   
1192508                -  ...       NaN        NaN          NaN   
1192509                T  ...       NaN        NaN        46:15   
1194274                A  ...       NaN        NaN        52:73   

        SangerRecalibWES_AC RNAseq_AC    HC_AC RD_AC WGS_AC  \
44657                 53:99       NaN      NaN   NaN    NaN   
46376                   NaN       7:2      NaN   NaN    NaN   
55016                 59:33     69:77   162:85   NaN    NaN   
61889                  9:29       NaN    53:84   NaN    NaN   
73171                 18:40    58:119  244:173   NaN    NaN   
...                     ...       ...      ...   ...    ...   
1177403               27:34       NaN      NaN   NaN    NaN   
1180341               65:88       NaN      NaN   NaN    NaN   
1192508               20:19       NaN      NaN   NaN    NaN   
1192509               61:79       NaN      NaN   NaN    NaN   
1194274               50:68       NaN      NaN   NaN    NaN   

        Variant_annotation   DepMap_ID  
44657             damaging  ACH-000136  
46376             damaging  ACH-000143  
55016             damaging  ACH-000174  
61889             damaging  ACH-000198  
73171             damaging  ACH-000233  
...                    ...         ...  
1177403           damaging  ACH-002255  
1180341           damaging  ACH-002256  
1192508           damaging  ACH-002272  
1192509           damaging  ACH-002272  
1194274           damaging  ACH-002273  

[87 rows x 35 columns]

In [93]:
sns.stripplot(col_colors,chromatin_profiling["H3K27me3K36me0"])

In [91]:
col_colors = muts_align["EZH2"]

subtype_palette = sns.color_palette(
    "Blues", len(col_colors.unique()))
subtype_color_map = dict(
    zip(col_colors.unique(), subtype_palette))
subtype_colors = col_colors.map(subtype_color_map)

subtype_colors

ACH-000001    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-000002    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-000004    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-000005    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-000006    (0.6718954248366014, 0.8143790849673203, 0.900...
                                    ...                        
ACH-001190    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-001306    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-001307    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-001318    (0.6718954248366014, 0.8143790849673203, 0.900...
ACH-001321    (0.6718954248366014, 0.8143790849673203, 0.900...
Name: EZH2, Length: 897, dtype: object

In [74]:
sns.clustermap(chromatin_align.fillna(chromatin_align.mean()).clip(-3,3).T,
               method="ward",
               cmap="bwr",
               col_colors=subtype_colors,
               figsize=(20,10)
              )